In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import re
import pickle

In [4]:
import collections
import math
import os
import random
import zipfile

In [2]:
with open('../data/namuwiki.json') as json_file:    
    data = json.load(json_file)

In [3]:
frame = pd.io.json.json_normalize(data)

## save pickle

In [15]:
f = open('../data/namuwiki.pick', 'wb')
pickle.dump(frame, f)
f.close()

## load pickle

In [6]:
f = open('../data/namuwiki.pick', 'rb')
frame = pickle.load(f)
f.close()

In [33]:
frame.head(3)

,contributors,namespace,text,title
0,"[namubot, R:hoon12560]",0,#redirect 느낌표\n,!
1,"[anatra95, chkong1998, Iviyuki, kirby10, max02...",0,[[파일:3444050440.jpg]]\n([[신 세계수의 미궁 2]]에서 뜬 !!...,!!아앗!!
2,"[ABC, AhnJ2000, aottkd3014, dream33, Fairy, ga...",0,"[include(틀:다른 뜻1, other1=말줄임표 등으로 사용하는 용어, rd1...",“……”


In [8]:
frame.shape

(842910, 4)

In [9]:
pat_redirect = re.compile('^#redirect (.+)')
def check_redirect(text):
    match = pat_redirect.match(text)
    if match:
        return match.group(1)
    else:
        return False

In [10]:
def get_article(title):
    p = frame.loc[frame['title'] == title]
    return p.text.values[0]    

In [92]:
pat_top = re.compile(r'\* 상위 문서 \: (.+)')
pat_bracket = re.compile(r'\[\[(.+?)\]\]')

def get_top(text):
    matches = pat_top.finditer(text)
    ret = []
    for match in matches:
        bracket = pat_bracket.search(match.group(1))
        if bracket:
            ret.append(bracket.group(1))
        else:
            ret.append(match.group(1))
    return ret

In [95]:
cate = ['배우/한국', '가수/한국']

In [106]:
actors = []
for idx, context in frame.iterrows():
    if any(actor in get_top(context.text) for actor in cate):
        actors.append(context.title)

In [109]:
print (len(actors))
for actor in actors[:5]:
    print (actor)

600
템플릿:배우
BeBe Mignon
BoA
KCM
KISUM


##  save actors

In [122]:
with open('../data/actors.txt', 'w') as f:
    for actor in actors:
        f.write(actor + r'\n')

In [34]:
pat_bracket = re.compile(r'\[\[(.+?)\]\]')
pat_file = re.compile(r'\[\[파일:(.+)\]\]')
pat_link = re.compile(r'\[\[(.+?)\|(.+?)\]\]')
pat_comment = re.compile(r'\[\*(.+?)\]')
pat_high = re.compile(r'\{\{\{(.+?)\}\}\}')

def bracket_filter(text):
    ret = ""
    match = pat_file.match(text)
    if match: 
        ret = ""
    else:
        match = pat_link.match(text)
        if match: 
            ret = match.group(2)
        else:
            ret = text[2:-2]
    return ret

def context_filter(text):
    # find bracket
    delc = 0
    matches = pat_bracket.finditer(text)
    for match in matches:
        conv = bracket_filter(match.group(0))
        text = text[:match.start() - delc] + conv +  text[match.end() - delc:]
        delc += len(match.group(0)) - len(conv)
        
    # comments
    delc = 0
    matches = pat_comment.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(0) +  text[match.end() - delc:]
        delc += 3
        
    # find highlight
    delc = 0
    matches = pat_high.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(1) +  text[match.end() - delc:]
        delc += 6
    
    return text

In [12]:
def tokenize(doc):
    return ['/'.join(t) for t in Twitter.pos(doc)]

In [26]:
def rawnize(title):
    return get_article(title)

In [27]:
def textnize(title):
    text = get_article(title)
    return context_filter(text)

In [28]:
print (rawnize('김태희'))

[include(틀:다른 뜻 설명1, subject=배우, other1=이외의 동명이인, rd1=김태희(동명이인))]
 * 상위 문서 : [[배우/한국]]
||||<table align=center><table width=560><table bordercolor=#32CD32><-2><:>[[파일:김태희.jpg|width=100%]]||
||<#32CD32> '''{{{#white 이름}}}''' || '''김태희(金泰希)''' ||
||<#32CD32> '''{{{#white 출생}}}''' || [[1980년]] [[3월 29일]], [[부산광역시]][* 출생당시에는 부산직할시였다.][* [[http://entertain.naver.com/read?oid=008&aid=0003295599|기사]]] ||
||<#32CD32> '''{{{#white 본관}}}''' || [[청풍]] [[김씨]] ||
||<#32CD32> '''{{{#white 성좌/지지}}}''' || [[양자리]]/[[원숭이띠]] ||
||<#32CD32> '''{{{#white 신체}}}''' || 162.5cm[* 2010년 9월 7일 방영된 [[김승우의 승승장구]]에서 직접 언급한 키는 162cm다. '많은 분들이 160cm도 안 되지 않냐' 고 오해를 한다며 이 부분만큼은 꼭 짚고 넘어가고 싶다고 했을 정도니 사실일 듯. 참고로 20대 한국여성 평균키가 160cm다.], 45kg, [[O형]] ||
||<#32CD32> '''{{{#white 학력}}}''' || 삼신초등학교[br]대현중학교[br]울산여자고등학교[br][[서울대학교]] 의류학 학사 ||
||<#32CD32> '''{{{#white 데뷔}}}''' || [[2000년]] CF [[화이트]] ||
||<#32CD32> '''{{{#white 가족}}}''' || 아버지 김유문, 어머니, 언니 김희원, 남동생 [[이완(배우)|이완]] ||
||<#32CD32> '''{{{#white 소속사}}}''' || 루아엔터테인먼

In [35]:
print (textnize('김태희'))

[include(틀:다른 뜻 설명1, subject=배우, other1=이외의 동명이인, rd1=김태희(동명이인))]
 * 상위 문서 : 배우/한국
||||<table align=center><table width=560><table bordercolor=#32CD32><-2><:>||
||<#32CD32> '''#white 이름''' || '''김태희(金泰希)''' ||
||<#32CD32> '''#white 출생''' || 1980년 3월 29일, 부산광역시[* 출생당시에는 부산직할시였[* 기사]기사] ||
||<#32CD32> '''#white 본관''' || 청풍 김씨 ||
||<#32CD32> '''#white 성좌/지지''' || 양자리/원숭이띠 ||
||<#32CD32> '''#white 신체''' || 1[* 2010년 9월 7일 방영된 김승우의 승승장구에서 직접 언급한 키는 162cm다. '많은 분들이 160cm도 안 되지 않냐' 고 오해를 한다며 이 부분만큼은 꼭 짚고 넘어가고 싶다고 했을 정도니 사실일 듯. 참고로 20대 한국여성 평균키가 160cm다.]0cm다.], 45kg, O형 ||
||<#32CD32> '''#white 학력''' || 삼신초등학교[br]대현중학교[br]울산여자고등학교[br]서울대학교 의류학 학사 ||
||<#32CD32> '''#white 데뷔''' || 2000년 CF 화이트 ||
||<#32CD32> '''#white 가족''' || 아버지 김유문, 어머니, 언니 김희원, 남동생 이완 ||
||<#32CD32> '''#white 소속사''' ||[* 김태희 형부 '정철우' 씨가 대표로 있는 가족 회사. '루아'는 '성령 또는 생명의 기운'이라는 의미.]'이라는 의미.][* 2016년 기준 前 카라 멤버인 강지영이 소속되어 있는 회사다.]속되어 있는 회사다.] # ||
||<#32CD32> '''#white 종교''' || 천주교(세례명: 베르다) ||
||<#32CD32> '''#white 팬 페이지''' || 

In [14]:
for idx, db in frame.iterrows():
    text = db.text
    chk = check_redirect(text)
    if chk:
        text = get_article(chk)
    context = context_filter(text)
    print (context)
    break

[include(틀:다른 뜻 넘어옴, name=!)]
 * 옛한글의 모음 - ᆝ
 * 수학에서의 느낌표 - 계승(팩토리얼)
 * MBC의 예능 프로그램 - 느낌표(MBC)
[include(틀:문장 부호)]
[include(틀:QWERTY 키보드)]
[목차]
== 개요 ==
감탄하거나 놀랐을때 쓰는 마침표. 반각문자로는 !이고 전각문자로는 ！이라고 쓴다

== 유니코드로서의 특징 ==
아스키 코드 및 유니코드 기준으로 공백을 제외하면 가장 앞에 위치해 있는 글자다(U+0021). 물론 코드값상에서는 맨 앞자리는 아니긴 하나, 제어문자라는 특성상 일반적인 '''"글자"'''로 치지 않기 때문에 실질적으로는 느낌표가 맨 앞이라고 보면 된다.

== 문장 부호로서의 특징 ==
물음표와 마찬가지로 언어에 따라 몇가지 베리에이션이 있는데, 스페인어에서는 감탄문을 시작할때 거꾸로된 느낌표(¡)도 붙인다. ~~알파벳 i가 아니다!~~[* 일부에서는 i를 !로 쓰는 경우도 있다. (예시: Mirror -> M!rror, Tooniverse -> Toon!verse), pink->P!NK(가수이다)]

ex> ¡Qué chica tan guapa! (얼마나 아름다운 소녀인가!)

사실 느낌표는 물음표에 비해 사용용도가 꽤 다양한 편이다. 언어적인 용도로도 꽤 많이 다양하게 이용되는 편인데 예를 들자면 독일어에선 편지의 인사말 또는 명령문에 느낌표를 붙인다! 국제음성기호에서는 치경 흡착음(alveolar clicks)을 나타내는 기호로 쓰인다. 혀를 찰 때 내는 '쯧쯧쯧……'의 실제 발음이 이거다! 몇몇 서적들을 보면 볼수 있는데 예를 들어, 우리가 흔히 '부시맨' 이라고 알고 있는 아프리카의 부족이름은 '!쿵족'이다.

양판소나 인터넷 소설에서 장음 표기나 강조를 위해 여러개를 이어 쓰는 경우도 있지만, 올바른 사용법이 아닌데다 보기에도 좋지 않다.
 * 예시> 죽어라!!!!!!!!!![* 참고로 소설속의 느낌표나 물음표는 문장의 끝에만 오기 때문에 2개 이상을 한꺼번에 쓸 수